In [19]:
import numpy as np 
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
import random
# from mtranslate import translate
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Concatenate, Conv2D, MaxPooling2D, Dropout,GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import random
from tqdm import tqdm


In [20]:
import pickle
image_data = np.array(pickle.load(open('/kaggle/input/image-data/image_data.pkl', 'rb')))
y = pickle.load(open('/kaggle/input/target/y.pkl', 'rb'))
for i in range(3,6):
    new = np.array(pickle.load(open(f'/kaggle/input/image-data/image_data_{i}.pkl', 'rb')))
    image_data = np.concatenate((image_data, new), axis=0)
    y_n = np.array(pickle.load(open(f'/kaggle/input/target/y_{i}.pkl', 'rb')))
    y = np.concatenate((y, y_n), axis=0)

In [21]:
import gc
del new
gc.collect()

21137

In [22]:
os.environ['PYTHONHASHSEED']=str(1)
tf.random.set_seed(1)
np.random.seed(1)
random.seed(1)
import warnings
warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
# Combine text and image data
inputs_image = Input(shape=(224, 224, 3))

image_model = Conv2D(32, (3, 3), activation='relu')(inputs_image)
image_model = MaxPooling2D((2, 2))(image_model)
image_model = Conv2D(64, (3, 3), activation='relu')(image_model)
image_model = MaxPooling2D((2, 2))(image_model)
image_model = Conv2D(128, (3, 3), activation='relu')(image_model)
image_model = MaxPooling2D((2, 2))(image_model)
image_model = Flatten()(image_model)
image_model = Dense(256, activation='relu')(image_model)
image_model = Dropout(0.5)(image_model)
output = Dense(1, activation='sigmoid')(image_model)

# Build and compile the model
model = Model(inputs=inputs_image, outputs=output)
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Split data into train and test sets
X_image_train, X_image_test, y_train, y_test = train_test_split(image_data, y, test_size=0.2, random_state=42)

# Train the model
X_image_train = np.array(X_image_train)
y_train = np.array(y_train)
with tf.device('/GPU:0'):
    model.fit(X_image_train, y_train, batch_size=64, epochs=14, validation_split = 0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_image_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy) #0.8864


Epoch 1/14
110/110 ━━━━━━━━━━━━━━━━━━━━ 14s 91ms/step - accuracy: 0.7453 - loss: 1.1669 - val_accuracy: 0.7710 - val_loss: 0.5061
Epoch 2/14
110/110 ━━━━━━━━━━━━━━━━━━━━ 7s 61ms/step - accuracy: 0.7998 - loss: 0.4479 - val_accuracy: 0.8591 - val_loss: 0.3737
Epoch 3/14
110/110 ━━━━━━━━━━━━━━━━━━━━ 7s 61ms/step - accuracy: 0.8663 - loss: 0.3281 - val_accuracy: 0.8750 - val_loss: 0.3353
Epoch 4/14
110/110 ━━━━━━━━━━━━━━━━━━━━ 7s 62ms/step - accuracy: 0.8947 - loss: 0.2609 - val_accuracy: 0.8716 - val_loss: 0.3691
Epoch 5/14
110/110 ━━━━━━━━━━━━━━━━━━━━ 10s 61ms/step - accuracy: 0.9163 - loss: 0.2123 - val_accuracy: 0.8591 - val_loss: 0.4931
Epoch 6/14
110/110 ━━━━━━━━━━━━━━━━━━━━ 7s 61ms/step - accuracy: 0.9343 - loss: 0.1780 - val_accuracy: 0.8665 - val_loss: 0.4768
Epoch 7/14
110/110 ━━━━━━━━━━━━━━━━━━━━ 7s 61ms/step - accuracy: 0.9541 - loss: 0.1184 - val_accuracy: 0.8710 - val_loss: 0.5212
Epoch 8/14
110/110 ━━━━━━━━━━━━━━━━━━━━ 7s 61ms/step - accuracy: 0.9644 - loss: 0.0975 - val_ac

In [23]:
model.save_weights('im_weights.weights.h5')